<a href="https://colab.research.google.com/github/True-star-580/final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset/blob/main/final_multimal_msa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# # RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

msa_challenge_at_the_4th_pazhou_ai_competition_path = kagglehub.competition_download('msa-challenge-at-the-4th-pazhou-ai-competition')
paulnkamau_dataset_cmu_mosei_extracted_path = kagglehub.dataset_download('paulnkamau/dataset-cmu-mosei-extracted')
paulnkamau_dataset_ch_simsv2_reorganized_path = kagglehub.dataset_download('paulnkamau/dataset-ch-simsv2-reorganized')
paulnkamau_dataset_public_test_n_validation_msa_v_to_a_to_t_path = kagglehub.dataset_download('paulnkamau/dataset-public-test-n-validation-msa-v-to-a-to-t')
paulnkamau_dataset_msa_translations_complete_path = kagglehub.dataset_download('paulnkamau/dataset-msa-translations-complete')

print('Data source import complete.')


100%|██████████| 24.7G/24.7G [14:18<00:00, 30.8MB/s]

Extracting files...


In [ ]:
import os
import pandas as pd
import json

def preview_directory(path, name):
    print(f"--- Previewing {name} ({path}) ---")
    if not os.path.exists(path):
        print(f"Directory not found: {path}")
        return

    file_list = []
    for root, _, files in os.walk(path):
        for file in files:
            file_list.append(os.path.join(root, file))

    if not file_list:
        print("No files found in the directory.")
        return

    print(f"Total number of files: {len(file_list)}")
    print("First 5 files:")
    for i, file_path in enumerate(file_list[:5]):
        print(file_path)

    print("\nPreviews of text-based files:")
    previewed_count = 0
    for file_path in file_list:
        if file_path.lower().endswith(('.csv', '.json')):
            try:
                print(f"\n--- {file_path} ---")
                if file_path.lower().endswith('.csv'):
                    # Try reading with different encodings
                    try:
                        df = pd.read_csv(file_path, encoding='utf-8')
                    except UnicodeDecodeError:
                        try:
                            df = pd.read_csv(file_path, encoding='latin-1')
                        except Exception as e:
                            print(f"Could not read CSV with common encodings: {e}")
                            continue
                    display(df.head())
                elif file_path.lower().endswith('.json'):
                    with open(file_path, 'r', encoding='utf-8') as f:
                        try:
                            data = json.load(f)
                            # Print first few items or keys for preview
                            if isinstance(data, list):
                                display(data[:5])
                            elif isinstance(data, dict):
                                display(list(data.keys())[:5])
                            else:
                                print(str(data)[:500] + '...') # Print first 500 chars
                        except Exception as e:
                            print(f"Could not read or parse JSON: {e}")

                previewed_count += 1
                if previewed_count >= 3: # Limit text file previews
                    print("\nLimiting text file previews to the first 3 found.")
                    break
            except Exception as e:
                print(f"Error processing {file_path}: {e}")


preview_directory(msa_challenge_at_the_4th_pazhou_ai_competition_path, 'msa-challenge-at-the-4th-pazhou-ai-competition')
preview_directory(paulnkamau_dataset_cmu_mosei_extracted_path, 'paulnkamau/dataset-cmu-mosei-extracted')
preview_directory(paulnkamau_dataset_ch_simsv2_reorganized_path, 'paulnkamau/dataset-ch-simsv2-reorganized')
preview_directory(paulnkamau_dataset_public_test_n_validation_msa_v_to_a_to_t_path, 'paulnkamau/dataset-public-test-n-validation-msa-v-to-a-to-t')
preview_directory(paulnkamau_dataset_msa_translations_complete_path, 'paulnkamau/dataset-msa-translations-complete')
preview_directory(paulnkamau_bert_mosei_path, 'paulnkamau/bert-mosei')
preview_directory(paulnkamau_notebookd03142330b_path, 'paulnkamau/notebookd03142330b') # for sample submission


In [ ]:
#

In [ ]:
!git clone https://github.com/True-star-580/final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset.git
%cd final_Multimodal-Sentiment-Analysis-with-MOSEI-Dataset

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
!pip install -q langdetect

In [ ]:
# !python main.py

In [ ]:
# we had trained the model earlier, its here '/content/multimodal_fusion_best.pt' or any path you choose to place it. model weights attached in the email

In [ ]:
!python inference.py

In [ ]:
# import pandas as pd
# pd.read_csv('/content/predictions.csv')

In [ ]:
msa_challenge_at_the_4th_pazhou_ai_competition_path

In [ ]:
import os

video_directory = '/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/Validation_Data/Validation_Data'
video_extensions = ['.mp4', '.avi', '.mov', '.mkv'] # Add other video extensions if needed

video_count = 0
if os.path.isdir(video_directory):
    for filename in os.listdir(video_directory):
        if any(filename.lower().endswith(ext) for ext in video_extensions):
            video_count += 1
    print(f"Number of video files in {video_directory}: {video_count}")
else:
    print(f"Directory not found: {video_directory}")

vids are chinese that i uploaded, so model doesnt perform very well, we move to step 2.

In [ ]:
%%writefile inference_two.py
# inference.py
import os
import sys
import logging
from pathlib import Path
import torch
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, pipeline
from moviepy.editor import VideoFileClip
import re # Import regex for character detection

# Add project root to path for our model imports
sys.path.append(str(Path(__file__).resolve().parent))

from src.models.fusion import TransformerFusionModel
from config import (
    TEXT_EMBEDDING_DIM, HIDDEN_DIM, NUM_ATTENTION_HEADS,
    NUM_TRANSFORMER_LAYERS, DROPOUT_RATE
)

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# --- Language Detection and Translation Functions ---
def contains_cjk(text):
    """Checks if the text contains any CJK (Chinese, Japanese, Korean) characters."""
    if not text or not text.strip():
        return False
    # CJK Unified Ideographs range: \u4e00-\u9fff
    return bool(re.search(r'[\u4e00-\u9fff]', text))

def translate_chinese_to_english(text, translator):
    """Translates Chinese text to English."""
    try:
        logger.info("Translating Chinese text to English...")
        result = translator(text)
        translated_text = result[0]['translation_text']
        logger.info(f"Translated text: '{translated_text[:100]}...'")
        return translated_text
    except Exception as e:
        logger.error(f"Translation failed: {e}")
        return text  # Return original text if translation fails

# --- Feature Extraction Functions ---

def extract_audio_from_video(video_path, audio_output_path):
    """Extracts audio from a video file and saves it as WAV."""
    try:
        logger.info(f"Extracting audio from {video_path}...")
        video_clip = VideoFileClip(str(video_path))
        audio_clip = video_clip.audio
        if audio_clip is None:
            logger.error(f"No audio track found in {video_path}")
            return False
        audio_clip.write_audiofile(str(audio_output_path), codec='pcm_s16le', logger=None)
        video_clip.close()
        logger.info(f"Audio saved to {audio_output_path}")
        return True
    except Exception as e:
        logger.error(f"Failed to extract audio from {video_path}: {e}")
        return False

def get_transcript_from_audio(audio_path, device):
    """Generates a transcript from an audio file using Whisper."""
    try:
        logger.info(f"Transcribing audio from {audio_path}...")
        pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)
        result = pipe(str(audio_path))
        transcript = result['text'].strip()
        logger.info(f"Transcript: '{transcript[:100]}...'")
        return transcript
    except Exception as e:
        logger.error(f"Failed to transcribe audio {audio_path}: {e}")
        return ""

def get_text_embedding(text, tokenizer, bert_model, device):
    """Converts raw text to a BERT [CLS] embedding."""
    try:
        inputs = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].cpu()
        return embedding
    except Exception as e:
        logger.error(f"Failed to create text embedding: {e}")
        return torch.zeros(1, TEXT_EMBEDDING_DIM)

# --- Main Inference Logic ---

def run_inference():
    """
    Main function to run inference on a directory of video files.
    """
    # 1. Get user input
    model_path_str = input("Enter the path to your trained model checkpoint (.pt): ").strip()
    video_dir_str = input("Enter the path to the directory containing your .mp4 files: ").strip()

    model_path = Path(model_path_str)
    video_dir = Path(video_dir_str)

    if not model_path.exists():
        logger.error(f"Model checkpoint not found: {model_path}")
        return
    if not video_dir.is_dir():
        logger.error(f"Video directory not found: {video_dir}")
        return

    # 2. Setup Device
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {DEVICE}")

    # 3. Load the trained model
    logger.info("Loading trained model...")
    model = TransformerFusionModel(
        text_dim=TEXT_EMBEDDING_DIM,
        audio_dim=0,
        visual_dim=0,
        hidden_dim=HIDDEN_DIM,
        num_layers=NUM_TRANSFORMER_LAYERS,
        num_heads=NUM_ATTENTION_HEADS,
        dropout_rate=DROPOUT_RATE
    )
    checkpoint = torch.load(model_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(DEVICE)
    model.eval()
    logger.info("Model loaded successfully.")

    # 4. Load the feature extractor (BERT)
    logger.info("Loading BERT model for feature extraction...")
    model_name = "bert-base-multilingual-cased"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    bert_model = BertModel.from_pretrained(model_name).to(DEVICE)
    bert_model.eval()

    # 5. Load the translation pipeline for Chinese to English
    logger.info("Loading translation model...")
    translator = pipeline(
        "translation",
        model="Helsinki-NLP/opus-mt-zh-en",
        device=DEVICE
    )
    logger.info("Translation model loaded successfully.")

    # 6. Process each video file
    video_files = list(video_dir.glob("*.mp4"))
    if not video_files:
        logger.error(f"No .mp4 files found in {video_dir}")
        return

    predictions_data = []
    temp_dir = Path("./temp_audio")
    temp_dir.mkdir(exist_ok=True)

    for video_path in tqdm(video_files, desc="Processing Videos"):
        video_id = video_path.stem
        temp_audio_path = temp_dir / f"{video_id}.wav"

        if not extract_audio_from_video(video_path, temp_audio_path):
            continue

        transcript = get_transcript_from_audio(temp_audio_path, DEVICE)
        if not transcript:
            logger.warning(f"Skipping video {video_id} due to empty transcript.")
            os.remove(temp_audio_path)
            continue

        sentiment_original = None
        sentiment_translated = None
        processed_text = transcript # Default processed text is the original

        if contains_cjk(transcript):
            logger.info("CJK characters detected. Processing as Chinese and translating.")

            # 1. Get sentiment for the original Chinese text
            original_embedding = get_text_embedding(transcript, tokenizer, bert_model, DEVICE)
            model_input_orig = {"language": original_embedding.to(DEVICE)}
            with torch.no_grad():
                prediction_orig = model(model_input_orig)
                sentiment_original = prediction_orig.item()

            # 2. Translate and get sentiment for the English text
            processed_text = translate_chinese_to_english(transcript, translator)
            translated_embedding = get_text_embedding(processed_text, tokenizer, bert_model, DEVICE)
            model_input_trans = {"language": translated_embedding.to(DEVICE)}
            with torch.no_grad():
                prediction_trans = model(model_input_trans)
                sentiment_translated = prediction_trans.item()

            logger.info(f"Video: {video_path.name}, Original Sentiment: {sentiment_original:.4f}, Translated Sentiment: {sentiment_translated:.4f}")

        else:
            logger.info("No CJK characters detected. Processing as English.")
            # Only one sentiment score is needed
            embedding = get_text_embedding(transcript, tokenizer, bert_model, DEVICE)
            model_input = {"language": embedding.to(DEVICE)}
            with torch.no_grad():
                prediction = model(model_input)
                sentiment_original = prediction.item()

            logger.info(f"Video: {video_path.name}, Predicted Sentiment: {sentiment_original:.4f}")

        predictions_data.append({
            "ID": video_path.name,
            "Sentiment_Original": sentiment_original,
            "Sentiment_Translated": sentiment_translated,
            "Original_Transcript": transcript,
            "Processed_Text": processed_text
        })


        os.remove(temp_audio_path)

    # 7. Save results to CSV
    if predictions_data:
        df = pd.DataFrame(predictions_data)
        output_csv_path = video_dir / "predictions.csv"
        df.to_csv(output_csv_path, index=False)
        logger.info(f"Inference complete! Predictions saved to {output_csv_path}")
        print(f"INFERENCE COMPLETE! PREDICTIONS SAVED TO {output_csv_path}")
    else:
        logger.warning("No predictions were made.")

    os.rmdir(temp_dir)

if __name__ == "__main__":
    run_inference()

In [ ]:
!python inference_two.py

In [ ]:
pd.read_csv('/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/Validation_Data/Validation_Data/predictions.csv')

In [ ]:
pd.read_csv('/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/validation_real_labels.csv')

In [ ]:
#####

In [ ]:
from pathlib import Path
VAL_DATA_PATH = Path("/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/validation_real_labels.csv")
pd.read_csv(VAL_DATA_PATH)

In [ ]:
VAL_TEXT_PATH = Path("/root/.cache/kagglehub/datasets/paulnkamau/dataset-public-test-n-validation-msa-v-to-a-to-t/versions/1/transcribed_text/validation")
#example; they are all in chinese tho.
!head /root/.cache/kagglehub/datasets/paulnkamau/dataset-public-test-n-validation-msa-v-to-a-to-t/versions/1/transcribed_text/validation/validate_video_001.txt


In [ ]:
%%writefile preprocess_v2.py
# preprocess_v2.py
import pandas as pd
from pathlib import Path
import logging
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# --- Define Paths ---
# Input paths for the datasets
CH_SIMS_PATH = Path("/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/CH-SIMSv2/ch-simsv2s/meta.csv")
CMU_MOSEI_PATH = Path("/root/.cache/kagglehub/datasets/paulnkamau/dataset-msa-translations-complete/versions/1/CMU_Mosei_English/cmu_mosei_bilingual_en_zh.csv")
VAL_DATA_PATH = Path("/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/validation_real_labels.csv")
VAL_TEXT_PATH = Path("/root/.cache/kagglehub/datasets/paulnkamau/dataset-public-test-n-validation-msa-v-to-a-to-t/versions/1/transcribed_text/validation")

# Output directory for the processed file
OUTPUT_DIR = Path("./processed_data_v2")
OUTPUT_DIR.mkdir(exist_ok=True)

# --- Define Label Mappings ---
LABEL_MAP_STR_TO_INT = {'SNEG': 0, 'WNEG': 1, 'NEUT': 2, 'WPOS': 3, 'SPOS': 4}

def map_ch_sims_label(score):
    """Maps numerical scores from CH-SIMS v2 to string labels."""
    if score in [-1.0, -0.8]: return 'SNEG'
    if score in [-0.6, -0.4, -0.2]: return 'WNEG'
    if score == 0.0: return 'NEUT'
    if score in [0.2, 0.4, 0.6]: return 'WPOS'
    if score in [0.8, 1.0]: return 'SPOS'
    return None

# --- Data Processing Functions ---

def process_ch_sims():
    """Processes the CH-SIMS v2 dataset."""
    logging.info(f"Processing CH-SIMS v2 from {CH_SIMS_PATH}...")
    df = pd.read_csv(CH_SIMS_PATH)
    df = df[['text', 'label', 'mode']].copy()
    df.rename(columns={'mode': 'split'}, inplace=True)
    df['label_str'] = df['label'].apply(map_ch_sims_label)
    df['language'] = 'chinese'
    df = df.dropna(subset=['label_str'])
    logging.info(f"Found {len(df)} samples in CH-SIMS v2.")
    return df[['text', 'label_str', 'language', 'split']]

def process_cmu_mosei():
    """Processes the CMU-MOSEI dataset."""
    logging.info(f"Processing CMU-MOSEI from {CMU_MOSEI_PATH}...")
    df = pd.read_csv(CMU_MOSEI_PATH)
    # Use the corrected English text for consistency
    df = df[['original_english', 'Label', 'split']].copy()
    df.rename(columns={'original_english': 'text', 'Label': 'label_str'}, inplace=True)
    df['language'] = 'english'
    df = df.dropna(subset=['text', 'label_str'])
    # Standardize 'valid' split name to 'val' for consistency
    df['split'] = df['split'].replace('valid', 'val')
    logging.info(f"Found {len(df)} samples in CMU-MOSEI.")
    return df

def process_validation_data():
    """
    Processes the competition's validation data.
    Reads labels and corresponding text files, then splits 90% for training
    and 10% for validation.
    """
    logging.info(f"Processing validation data from {VAL_DATA_PATH} and {VAL_TEXT_PATH}...")
    df_labels = pd.read_csv(VAL_DATA_PATH)

    texts = []
    for _, row in df_labels.iterrows():
        video_id = row['ID']
        text_filename = video_id.replace('.mp4', '.txt')
        text_file_path = VAL_TEXT_PATH / text_filename

        try:
            with open(text_file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
                texts.append(text)
        except FileNotFoundError:
            logging.warning(f"Text file not found for {video_id}, skipping.")
            texts.append(None)

    df_labels['text'] = texts
    df_labels = df_labels.dropna(subset=['text']) # Drop rows where text file was not found

    df_labels.rename(columns={'Label': 'label_str'}, inplace=True)
    df_labels['language'] = 'chinese'

    # Keep only necessary columns for splitting
    df = df_labels[['text', 'label_str', 'language']].copy()

    # Split the data: 90% for training, 10% for validation
    df_train, df_val = train_test_split(
        df,
        test_size=0.1,
        random_state=42,
        stratify=df['label_str'] # Ensure label distribution is maintained
    )

    df_train['split'] = 'train'
    df_val['split'] = 'val'

    df_split = pd.concat([df_train, df_val], ignore_index=True)

    logging.info(f"Processed {len(df_split)} samples from validation data. Split into {len(df_train)} train and {len(df_val)} val samples.")
    return df_split

def run():
    """Main function to run the entire preprocessing pipeline."""
    # Process each dataset individually
    df_ch = process_ch_sims()
    df_mosei = process_cmu_mosei()
    df_val_split = process_validation_data()

    # Combine all three dataframes
    df_combined = pd.concat([df_ch, df_mosei, df_val_split], ignore_index=True)

    # Shuffle the entire dataset to mix data from all sources
    df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

    # Apply the final integer mapping to the label
    df_combined['label'] = df_combined['label_str'].map(LABEL_MAP_STR_TO_INT)

    # Final cleanup: ensure no missing labels and correct data type
    df_combined = df_combined.dropna(subset=['label'])
    df_combined['label'] = df_combined['label'].astype(int)

    # Save the final unified dataset
    output_path = OUTPUT_DIR / "finetuning_data.csv"
    df_combined.to_csv(output_path, index=False)

    # Log summary statistics
    logging.info(f"--- Preprocessing Complete ---")
    logging.info(f"Unified dataset created with {len(df_combined)} samples.")
    logging.info(f"Saved to {output_path}")
    logging.info("\nValue counts for labels (string):\n" + str(df_combined['label_str'].value_counts()))
    logging.info("\nValue counts for final splits:\n" + str(df_combined['split'].value_counts()))

if __name__ == '__main__':
    run()

In [ ]:
!python preprocess_v2.py

In [ ]:
%%writefile main_v2.py
# main_v2.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
from functools import partial

# Import our new V2 modules
import preprocess_v2
from src.data.dataset_v2 import FinetuneDataset, collate_fn
from src.models.fusion_v2 import MultilingualSentimentClassifier
from src.training.trainer_v2 import TrainerV2
from config import HIDDEN_DIM

def run_finetuning():
    # --- Config ---
    MODEL_NAME = "google-bert/bert-base-multilingual-cased"
    NUM_EPOCHS = 7
    LEARNING_RATE = 2e-5
    BATCH_SIZE = 64
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    print(f"Using device: {DEVICE}")

    # --- 1. Load Tokenizer and Pre-trained BERT ---
    print(f"Loading tokenizer and model for '{MODEL_NAME}'...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    bert_model = AutoModel.from_pretrained(MODEL_NAME)

    # --- 2. Create Datasets and DataLoaders ---
    print("Creating datasets...")
    train_dataset = FinetuneDataset(split='train')
    val_dataset = FinetuneDataset(split='val')

    # Use a partial function to pass the tokenizer to the collate_fn
    collate_with_tokenizer = partial(collate_fn, tokenizer=tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_with_tokenizer)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_with_tokenizer)

    # --- 3. Initialize Model ---
    print("Initializing classification model...")
    model = MultilingualSentimentClassifier(bert_model, hidden_dim=HIDDEN_DIM, num_classes=5)
    model.to(DEVICE)

    # --- 4. Setup Optimizer and Loss ---
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    # --- 5. Train ---
    trainer = TrainerV2(model, optimizer, criterion, DEVICE, train_loader, val_loader)
    print("Starting fine-tuning...")
    trainer.train(num_epochs=NUM_EPOCHS)

    # --- 6. Save Model ---
    torch.save(model.state_dict(), './models/finetuned_multilingual_v2.pt')
    print("Fine-tuning complete. Model saved to './models/finetuned_multilingual_v2.pt'")

def main():
    print("--- Multilingual Fine-tuning Pipeline V2 ---")
    print("1. Preprocess and unify datasets")
    print("2. Run fine-tuning on the unified dataset")
    choice = input("Enter your choice (1-2): ").strip()

    if choice == '1':
        preprocess_v2.run()
    elif choice == '2':
        run_finetuning()
    else:
        print("Invalid choice.")

if __name__ == "__main__":
    main()

In [ ]:
# pd.read_csv('/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/CH-SIMSv2/ch-simsv2s/meta.csv')

In [ ]:
# pd.read_csv('/root/.cache/kagglehub/datasets/paulnkamau/dataset-msa-translations-complete/versions/1/CMU_Mosei_English/cmu_mosei_bilingual_en_zh.csv')

In [ ]:
!python preprocess_v2.py

In [ ]:
!python main_v2.py

In [ ]:
%%writefile inference_dual.py
# inference_dual.py
import os
import sys
import logging
from pathlib import Path
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, pipeline
from moviepy.editor import VideoFileClip


sys.path.append(str(Path(__file__).resolve().parent))

# Import BOTH model architectures
from src.models.fusion import TransformerFusionModel as ModelV1
from src.models.fusion_v2 import MultilingualSentimentClassifier as ModelV2
from config import (
    TEXT_EMBEDDING_DIM, HIDDEN_DIM, NUM_ATTENTION_HEADS,
    NUM_TRANSFORMER_LAYERS, DROPOUT_RATE
)

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# --- Constants and Mappings ---
LABEL_INT_TO_STR_V2 = {0: 'SNEG', 1: 'WNEG', 2: 'NEUT', 3: 'WPOS', 4: 'SPOS'}

def score_to_class_v1(score):
    """Converts a V1 regression score to a V2 class string."""
    if -3.0 <= score <= -2.0: return 'SNEG'
    if -2.0 < score <= -0.5: return 'WNEG'
    if -0.5 < score < 0.5: return 'NEUT'
    if 0.5 <= score < 2.0: return 'WPOS'
    if 2.0 <= score <= 3.0: return 'SPOS'
    return 'NEUT' # Default case


def extract_audio_from_video(video_path, audio_output_path):
    try:
        video_clip = VideoFileClip(str(video_path))
        audio_clip = video_clip.audio
        if audio_clip is None: return False
        audio_clip.write_audiofile(str(audio_output_path), codec='pcm_s16le', logger=None)
        video_clip.close()
        return True
    except Exception: return False

def get_transcript_and_language(audio_path, device):
    """Generates transcript and detects language using Whisper."""
    try:
        pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)
        # The 'return_timestamps' flag can provide more metadata, including language
        result = pipe(str(audio_path), generate_kwargs={"language": None}, return_timestamps=False)
        transcript = result['text'].strip()

        # Whisper's pipeline doesn't directly expose language in the simple output,
        # but we can infer it. For simplicity, we'll check for Chinese characters.
        language = 'chinese' if any('\u4e00' <= char <= '\u9fff' for char in transcript) else 'english'

        logger.info(f"Detected Language: {language} | Transcript: '{transcript[:50]}...'")
        return transcript, language
    except Exception as e:
        logger.error(f"Failed to transcribe audio {audio_path}: {e}")
        return "", "unknown"

def get_text_embedding(text, tokenizer, bert_model, device):
    try:
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}
        with torch.no_grad():
            outputs = bert_model(**inputs)
            embedding = outputs.last_hidden_state
        return embedding
    except Exception: return torch.zeros(1, 512, TEXT_EMBEDDING_DIM)

# --- Main Inference Logic ---

def run_dual_inference():
    """
    Main function to run hybrid inference using both V1 and V2 models.
    """
    # 1. Get user input
    model_v1_path_str = input("Enter path to the Multimodal Fusion Model (V1, .pt): ").strip()
    model_v2_path_str = input("Enter path to your FINETUNED Multilingual model (V2, .pt): ").strip()
    video_dir_str = input("Enter path to the directory with the .mp4 files: ").strip()

    model_v1_path = Path(model_v1_path_str)
    model_v2_path = Path(model_v2_path_str)
    video_dir = Path(video_dir_str)

    # Validate paths
    if not model_v1_path.exists() or not model_v2_path.exists() or not video_dir.is_dir():
        logger.error("One or more paths are invalid. Please check and try again.")
        return

    # 2. Setup Device and Feature Extractor
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Using device: {DEVICE}")

    logger.info("Loading Multilingual BERT for feature extraction...")
    bert_model_name = "bert-base-multilingual-cased"
    tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
    feature_extractor_bert = AutoModel.from_pretrained(bert_model_name).to(DEVICE)
    feature_extractor_bert.eval()

    # 3. Load Models
    logger.info("Loading Model V1 (Multimodal Fusion)...")
    model_v1 = ModelV1(
        text_dim=TEXT_EMBEDDING_DIM, audio_dim=0, visual_dim=0, hidden_dim=HIDDEN_DIM,
        num_layers=NUM_TRANSFORMER_LAYERS, num_heads=NUM_ATTENTION_HEADS, dropout_rate=DROPOUT_RATE
    )

    logger.warning("Skipping V1 model loading due to embedding mismatch. Proceeding with V2-only logic.")

    logger.info("Loading Model V2 (Finetuned Multilingual)...")
    # First, create the model structure with the base BERT
    model_v2_structure = ModelV2(feature_extractor_bert, hidden_dim=HIDDEN_DIM, num_classes=5)
    # Then, load the fine-tuned weights
    model_v2_structure.load_state_dict(torch.load(model_v2_path, map_location=DEVICE))
    model_v2 = model_v2_structure.to(DEVICE)
    model_v2.eval()
    logger.info("Models loaded successfully.")

    # 4. Process each video file
    video_files = list(video_dir.glob("*.mp4"))
    predictions_data = []
    temp_dir = Path("./temp_audio_dual")
    temp_dir.mkdir(exist_ok=True)

    for video_path in tqdm(video_files, desc="Processing Videos"):
        video_id = video_path.stem
        temp_audio_path = temp_dir / f"{video_id}.wav"

        if not extract_audio_from_video(video_path, temp_audio_path):
            continue

        transcript, language = get_transcript_and_language(temp_audio_path, DEVICE)
        if not transcript:
            os.remove(temp_audio_path)
            continue

        # Prepare input for V2 model (tokenizer is part of the V2 pipeline)
        inputs = tokenizer(transcript, padding=True, truncation=True, max_length=512, return_tensors="pt")
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

        # Get V2 Prediction
        with torch.no_grad():
            logits = model_v2(**inputs)
            prediction_idx = torch.argmax(logits, dim=1).item()
            final_prediction_label = LABEL_INT_TO_STR_V2[prediction_idx]

        logger.info(f"Video: {video_path.name}, Language: {language}, Final Prediction: {final_prediction_label}")
        predictions_data.append({"ID": video_path.name, "Label": final_prediction_label})

        os.remove(temp_audio_path)

    # 5. Save results to CSV
    if predictions_data:
        df = pd.DataFrame(predictions_data)
        output_csv_path = video_dir / "predictions_dual.csv"
        df.to_csv(output_csv_path, index=False)
        logger.info(f"Inference complete! Predictions saved to {output_csv_path}")
        print(f"INFERENCE COMPLETE! Predictions saved to {output_csv_path}")

    # Clean up temp directory
    temp_dir.rmdir()


if __name__ == "__main__":
    run_dual_inference()

In [ ]:
/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/Public_Test_Data/Public_Test_Data

In [ ]:
!python inference_dual.py

In [ ]:
pd.read_csv('/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/Public_Test_Data/Public_Test_Data/predictions_dual.csv')

In [ ]:
from google.colab import files
files.download('/root/.cache/kagglehub/competitions/msa-challenge-at-the-4th-pazhou-ai-competition/Public_Test_Data/Public_Test_Data/predictions_dual.csv')
